In [2]:
# Import necessary libraries

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [3]:
%pip install neo4j-for-adk --quiet
%pip install neo4j --quiet

ERROR: Could not find a version that satisfies the requirement neo4j-for-adk (from versions: none)
ERROR: No matching distribution found for neo4j-for-adk
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb

In [5]:
# Comprehensive Neo4j Diagnostics
import os
import socket
import time
from neo4j import GraphDatabase

print("=== Neo4j Connection Diagnostics ===\n")

# 1. Check environment variables
print("1. Environment Variables:")
print(f"   NEO4J_URI: {os.getenv('NEO4J_URI', 'Not set')}")
print(f"   NEO4J_USERNAME: {os.getenv('NEO4J_USERNAME', 'Not set')}")
print(f"   NEO4J_PASSWORD: {'*' * len(os.getenv('NEO4J_PASSWORD', '')) if os.getenv('NEO4J_PASSWORD') else 'Not set'}")
print(f"   NEO4J_DATABASE: {os.getenv('NEO4J_DATABASE', 'Not set')}\n")

# 2. Check if ports are open
print("2. Port Connectivity Test:")
def check_port(host, port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except:
        return False

ports_to_check = [
    ('127.0.0.1', 7687, 'Neo4j Bolt (IPv4)'),
    ('localhost', 7687, 'Neo4j Bolt (localhost)'),
    ('127.0.0.1', 7474, 'Neo4j Browser (IPv4)'),
    ('localhost', 7474, 'Neo4j Browser (localhost)')
]

for host, port, desc in ports_to_check:
    is_open = check_port(host, port)
    status = "✅ OPEN" if is_open else "❌ CLOSED"
    print(f"   {desc} ({host}:{port}): {status}")

print()

# 3. Try different connection approaches
print("3. Connection Attempts:")

# Try with explicit IPv4
try:
    driver = GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "password123"))
    with driver.session() as session:
        result = session.run("RETURN 'IPv4 Direct' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ IPv4 Direct: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ IPv4 Direct: {type(e).__name__}: {str(e)[:100]}...")

# Try with localhost
try:
    driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password123"))
    with driver.session() as session:
        result = session.run("RETURN 'localhost' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ localhost: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ localhost: {type(e).__name__}: {str(e)[:100]}...")

# Try with environment variables
try:
    uri = os.getenv('NEO4J_URI', 'bolt://127.0.0.1:7687')
    username = os.getenv('NEO4J_USERNAME', 'neo4j')
    password = os.getenv('NEO4J_PASSWORD', 'password123')
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run("RETURN 'Env Vars' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ Environment Vars: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ Environment Vars: {type(e).__name__}: {str(e)[:100]}...")

print("\n=== Recommendations ===")
print("If all connection attempts fail:")
print("1. Wait 30-60 seconds after container start for Neo4j to fully initialize")
print("2. Check Docker logs: docker logs neo4j")
print("3. Verify container is running: docker ps | grep neo4j")
print("4. Try recreating container if issues persist:")
print("   docker rm -f neo4j")
print("   docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password123 neo4j:latest")

=== Neo4j Connection Diagnostics ===

1. Environment Variables:
   NEO4J_URI: bolt://127.0.0.1:7687
   NEO4J_USERNAME: neo4j
   NEO4J_PASSWORD: ***********
   NEO4J_DATABASE: neo4j

2. Port Connectivity Test:
   Neo4j Bolt (IPv4) (127.0.0.1:7687): ✅ OPEN
   Neo4j Bolt (localhost) (localhost:7687): ✅ OPEN
   Neo4j Browser (IPv4) (127.0.0.1:7474): ✅ OPEN
   Neo4j Browser (localhost) (localhost:7474): ✅ OPEN

3. Connection Attempts:


   ✅ IPv4 Direct: IPv4 Direct at 1768131129873
   ✅ localhost: localhost at 1768131129924
   ✅ Environment Vars: Env Vars at 1768131129944

=== Recommendations ===
If all connection attempts fail:
1. Wait 30-60 seconds after container start for Neo4j to fully initialize
2. Check Docker logs: docker logs neo4j
3. Verify container is running: docker ps | grep neo4j
4. Try recreating container if issues persist:
   docker rm -f neo4j
   docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password123 neo4j:latest


In [6]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message': 'Neo4j is Ready!'}]}


In [7]:
# Define Model Constants for easier use 
MODEL_GPT = "openai/gpt-5-nano"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", 
                                           "content": "Are you ready?"}], 
                                tools=[]))

print("\nOpenAI is ready for use.")

ModelResponse(id='chatcmpl-Cwnxy1c6L5BhkRMG74HXKgGM2zfXH', created=1768131130, model='gpt-5-nano-2025-08-07', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes—I'm ready. What would you like to do? I can help with answers, writing, coding, data analysis, planning, and more. If you have a goal or task in mind, share the details and I’ll propose steps or start right away.", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=254, prompt_tokens=107, total_tokens=361, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=192, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='

In [8]:
ner_agent_role_and_goal = """
You are a top-tier algorithm designer for analyzing text files and proposing
the kind of named entities that could be extracted which would be relevant 
for a user's goal.
"""

In [9]:
ner_agent_hints = """
Entities are people, places, things and qualities, but not quantities. 
Your goal is to propose a list of the type of entities, not the actual instances
of entities.

There are two general approaches to identifying types of entities:
- well-known entities: these closely correlate with approved node labels in an existing graph schema
- discovered entities: these may not exist in the graph schema, but appear consistently in the source text

Design rules for well-known entities:
- always use existing well-known entity types. For example, if there is a well-known type "Person", and people appear in the text, then propose "Person" as the type of entity.
- prefer reusing existing entity types rather than creating new ones

Design rules for discovered entities:
- discovered entities are consistently mentioned in the text and are highly relevant to the user's goal
- always look for entities that would provide more depth or breadth to the existing graph
- for example, if the user goal is to represent social communities and the graph has "Person" nodes, look through the text to discover entities that are relevant like "Hobby" or "Event"
- avoid quantitative types that may be better represented as a property on an existing entity or relationship.
- for example, do not propose "Age" as a type of entity. That is better represented as an additional property "age" on a "Person".
"""

In [10]:
ner_agent_chain_of_thought_directions = """
Prepare for the task:
- use the 'get_user_goal' tool to get the user goal
- use the 'get_approved_files' tool to get the list of approved files
- use the 'get_well_known_types' tool to get the approved node labels

Think step by step:
1. Sample some of the files using the 'sample_file' tool to understand the content
2. Consider what well-known entities are mentioned in the text
3. Discover entities that are frequently mentioned in the text that support the user's goal
4. Use the 'set_proposed_entities' tool to save the list of well-known and discovered entity types
5. Use the 'get_proposed_entities' tool to retrieve the proposed entities and present them to the user for their approval
6. If the user approves, use the 'approve_proposed_entities' tool to finalize the entity types
7. If the user does not approve, consider their feedback and iterate on the proposal
"""

In [11]:
ner_agent_instruction = f"""
{ner_agent_role_and_goal}
{ner_agent_hints}
{ner_agent_chain_of_thought_directions}
"""

#print(ner_agent_instruction)

In [12]:
# tools to propose and approve entity types
PROPOSED_ENTITIES = "proposed_entity_types"
APPROVED_ENTITIES = "approved_entity_types"

def set_proposed_entities(proposed_entity_types: list[str], tool_context:ToolContext) -> dict:
    """Sets the list proposed entity types to extract from unstructured text."""
    tool_context.state[PROPOSED_ENTITIES] = proposed_entity_types
    return tool_success(PROPOSED_ENTITIES, proposed_entity_types)

def get_proposed_entities(tool_context:ToolContext) -> dict:
    """Gets the list of proposed entity types to extract from unstructured text."""
    return tool_context.state.get(PROPOSED_ENTITIES, [])

def approve_proposed_entities(tool_context:ToolContext) -> dict:
    """Upon approval from user, records the proposed entity types as an approved list of entity types 

    Only call this tool if the user has explicitly approved the suggested files.
    """
    if PROPOSED_ENTITIES not in tool_context.state:
        return tool_error("No proposed entity types to approve. Please set proposed entities first, ask for user approval, then call this tool.")
    tool_context.state[APPROVED_ENTITIES] = tool_context.state.get(PROPOSED_ENTITIES)
    return tool_success(APPROVED_ENTITIES, tool_context.state[APPROVED_ENTITIES])

def get_approved_entities(tool_context:ToolContext) -> dict:
    """Get the approved list of entity types to extract from unstructured text."""
    return tool_context.state.get(APPROVED_ENTITIES, [])


In [13]:
def get_well_known_types(tool_context:ToolContext) -> dict:
    """Gets the approved labels that represent well-known entity types in the graph schema."""
    construction_plan = tool_context.state.get("approved_construction_plan", {})
    # approved labels are the keys for each construction plan entry where `construction_type` is "node"
    approved_labels = {entry["label"] for entry in construction_plan.values() if entry["construction_type"] == "node"}
    return tool_success("approved_labels", approved_labels)



In [14]:
from tools import get_approved_user_goal, get_approved_files, sample_file
ner_agent_tools = [
    get_approved_user_goal, get_approved_files, sample_file,
    get_well_known_types,
    set_proposed_entities,
    approve_proposed_entities
]

In [15]:
file_result = sample_file(file_path="product_reviews/gothenburg_table_reviews.md",tool_context=None)

print(file_result["content"])

# Gothenburg Table Reviews

Scraped from https://www.between2furns.com/Gothenburg-Table/dp/B0BQJWJWJW

## Rating: ★★★★★ (5/5)
This table is the centerpiece of our dining room! The modern design is stunning yet timeless, and the quality is exceptional. We've had it for 6 months now and it still looks brand new despite daily use with two kids. Assembly took about an hour with two people. The surface is easy to clean and surprisingly resistant to scratches. Highly recommend!

- @akollegger (Cambridge)

---

## Rating: ★★★☆☆ (3/5)
The Gothenburg Table looks beautiful, I'll give it that. But assembly was a nightmare - some of the pre-drilled holes didn't line up properly and I had to drill new ones. Once assembled, it's sturdy enough, but I expected better quality control for this price point. The table top also shows fingerprints very easily which is annoying for daily use.

- @furniture_lover92 (Seattle)

---

## Rating: ★★★★☆ (4/5)
love love LOVE the look of this table!! perfect size for

In [16]:
NER_AGENT_NAME = "ner_schema_agent_v1"
ner_schema_agent = Agent(
    name=NER_AGENT_NAME,
    description="Proposes the kind of named entities that could be extracted from text files.",
    model=llm,
    instruction=ner_agent_instruction,
    tools=ner_agent_tools, 
)


In [17]:
ner_agent_initial_state = {
    "approved_user_goal": {
        "kind_of_graph": "supply chain analysis",
        "description": """A multi-level bill of materials for manufactured products, useful for root cause analysis. 
        Add product reviews to start analysis from reported issues like quality, difficulty, or durability."""
    },
    "approved_files": [
        "product_reviews/gothenburg_table_reviews.md",
        "product_reviews/helsingborg_dresser_reviews.md",
        "product_reviews/jonkoping_coffee_table_reviews.md",
        "product_reviews/linkoping_bed_reviews.md",
        "product_reviews/malmo_desk_reviews.md",
        "product_reviews/norrkoping_nightstand_reviews.md",
        "product_reviews/orebro_lamp_reviews.md",
        "product_reviews/stockholm_chair_reviews.md",
        "product_reviews/uppsala_sofa_reviews.md",
        "product_reviews/vasteras_bookshelf_reviews.md"
    ],
    "approved_construction_plan": {
        "Product": {
            "construction_type": "node",
            "label": "Product",
        },
        "Assembly": {
            "construction_type": "node",
            "label": "Assembly",
        },
        "Part": {
            "construction_type": "node",
            "label": "Part",
        },
        "Supplier": {
            "construction_type": "node",
            "label": "Supplier",
        }
        # Relationship construction omitted, since it won't get used in this notebook
    }
}

In [18]:
from helper import make_agent_caller

ner_agent_caller = await make_agent_caller(ner_schema_agent, ner_agent_initial_state)

await ner_agent_caller.call("Add product reviews to the knowledge graph to trace product complaints back through the manufacturing process.")

# Alternatively, uncomment this line to get verbose output
# await ner_agent_caller.call("Add product reviews.", True)

session_end = await ner_agent_caller.get_session()

print("\n---\n")

print("\nSession state: ", session_end.state)

if PROPOSED_ENTITIES in session_end.state:
    print("\nProposed entities: ", session_end.state[PROPOSED_ENTITIES])

if APPROVED_ENTITIES in session_end.state:
    print("\nInappropriately approved entities: ", session_end.state[APPROVED_ENTITIES])
else:
    print("\nAwaiting approval.")


>>> User Query: Add product reviews to the knowledge graph to trace product complaints back through the manufacturing process.
<<< Agent Response: Thanks. Here’s a concise proposal of entity types to extract for adding product reviews to the knowledge graph to trace complaints through manufacturing.

What we already have in place
- Well-known (existing) entity types (to reuse in the graph):
  - Product
  - Assembly
  - Part
  - Supplier

- Discovered (new) entity types to add for depth and traceability:
  - Review
  - ReviewSource
  - Issue
  - IssueCategory
  - ProcessStep
  - Material
  - Finish

Rationale for these types
- Review: Represents individual customer feedback entries extracted from reviews. Captures rating, text, author, date, and source context.
- ReviewSource: Captures where the review came from (e.g., the review site or source platform). Helps trace provenance and assess source reliability.
- Issue: A specific problem or complaint described in reviews (e.g., “assembly

In [19]:
await ner_agent_caller.call("Approve the proposed entities.")

session_end = await ner_agent_caller.get_session()

ner_end_state = session_end.state if session_end else {}

print("Session state: ", ner_end_state)

if APPROVED_ENTITIES in ner_end_state:
    print("\nApproved entities: ", ner_end_state[APPROVED_ENTITIES])
else:
    print("\nStill awaiting approval? That is weird. Please check the agent's state and the proposed entities.")


>>> User Query: Approve the proposed entities.
<<< Agent Response: Approval recorded. The following entity types are now approved for extraction and graph integration:

- Product
- Assembly
- Part
- Supplier
- Review
- ReviewSource
- Issue
- IssueCategory
- ProcessStep
- Material
- Finish

What’s next (recommended plan)
- Define a concrete data model (properties and relationships):
  - Review: rating, text, author, date, source_ref
  - ReviewSource: name, url
  - Issue: description, severity/impact, notes
  - IssueCategory: category label (e.g., Assembly, Material, Finish)
  - ProcessStep: step name or stage (e.g., assembly, QC)
  - Material: material_name, material_type, specifications
  - Finish: finish_type, color, sheen
  - Product/Assembly/Part/Supplier: capture relevant attributes and interconnections (e.g., Part_of, Uses, Supplied_by)

- Pilot ingestion (pilot run):
  - Use a subset of approved files (e.g., Gothenburg Table Reviews and Linköping Bed Reviews) to demonstrate:
   

In [20]:
fact_agent_role_and_goal = """
  You are a top-tier algorithm designed for analyzing text files and proposing
  the type of facts that could be extracted from text that would be relevant 
  for a user's goal. 
"""

In [21]:
fact_agent_hints = """
  Do not propose specific individual facts, but instead propose the general type 
  of facts that would be relevant for the user's goal. 
  For example, do not propose "ABK likes coffee" but the general type of fact "Person likes Beverage".
  
  Facts are triplets of (subject, predicate, object) where the subject and object are
  approved entity types, and the proposed predicate provides information about
  how they are related. For example, a fact type could be (Person, likes, Beverage).

  Design rules for facts:
  - only use approved entity types as subjects or objects. Do not propose new types of entities
  - the proposed predicate should describe the relationship between the approved subject and object
  - the predicate should optimize for information that is relevant to the user's goal
  - the predicate must appear in the source text. Do not guess.
  - use the 'add_proposed_fact' tool to record each proposed fact type
"""

In [22]:
fact_agent_chain_of_thought_directions = """
    Prepare for the task:
    - use the 'get_approved_user_goal' tool to get the user goal
    - use the 'get_approved_files' tool to get the list of approved files
    - use the 'get_approved_entities' tool to get the list of approved entity types

    Think step by step:
    1. Use the 'get_approved_user_goal' tool to get the user goal
    2. Sample some of the approved files using the 'sample_file' tool to understand the content
    3. Consider how subjects and objects are related in the text
    4. Call the 'add_proposed_fact' tool for each type of fact you propose
    5. Use the 'get_proposed_facts' tool to retrieve all the proposed facts
    6. Present the proposed types of facts to the user, along with an explanation
"""

In [23]:
fact_agent_instruction = f"""
{fact_agent_role_and_goal}
{fact_agent_hints}
{fact_agent_chain_of_thought_directions}
"""

In [24]:
PROPOSED_FACTS = "proposed_fact_types"
APPROVED_FACTS = "approved_fact_types"

def add_proposed_fact(approved_subject_label:str,
                      proposed_predicate_label:str,
                      approved_object_label:str,
                      tool_context:ToolContext) -> dict:
    """Add a proposed type of fact that could be extracted from the files.

    A proposed fact type is a tuple of (subject, predicate, object) where
    the subject and object are approved entity types and the predicate 
    is a proposed relationship label.

    Args:
      approved_subject_label: approved label of the subject entity type
      proposed_predicate_label: label of the predicate
      approved_object_label: approved label of the object entity type
    """
    # Guard against invalid labels
    approved_entities = tool_context.state.get(APPROVED_ENTITIES, [])
    
    if approved_subject_label not in approved_entities:
        return tool_error(f"Approved subject label {approved_subject_label} not found. Try again.")
    if approved_object_label not in approved_entities:
        return tool_error(f"Approved object label {approved_object_label} not found. Try again.")
    
    current_predicates = tool_context.state.get(PROPOSED_FACTS, {})
    current_predicates[proposed_predicate_label] = {
        "subject_label": approved_subject_label,
        "predicate_label": proposed_predicate_label,
        "object_label": approved_object_label
    }
    tool_context.state[PROPOSED_FACTS] = current_predicates
    return tool_success(PROPOSED_FACTS, current_predicates)
    
def get_proposed_facts(tool_context:ToolContext) -> dict:
    """Get the proposed types of facts that could be extracted from the files."""
    return tool_context.state.get(PROPOSED_FACTS, {})


def approve_proposed_facts(tool_context:ToolContext) -> dict:
    """Upon user approval, records the proposed fact types as approved fact types

    Only call this tool if the user has explicitly approved the proposed fact types.
    """
    if PROPOSED_FACTS not in tool_context.state:
        return tool_error("No proposed fact types to approve. Please set proposed facts first, ask for user approval, then call this tool.")
    tool_context.state[APPROVED_FACTS] = tool_context.state.get(PROPOSED_FACTS)
    return tool_success(APPROVED_FACTS, tool_context.state[APPROVED_FACTS])


In [25]:
fact_agent_tools = [
    get_approved_user_goal, get_approved_files, 
    get_approved_entities,
    sample_file,
    add_proposed_fact,
    get_proposed_facts,
    approve_proposed_facts
]

In [26]:
FACT_AGENT_NAME = "fact_type_extraction_agent_v1"
relevant_fact_agent = Agent(
    name=FACT_AGENT_NAME,
    description="Proposes the kind of relevant facts that could be extracted from text files.",
    model=llm,
    instruction=fact_agent_instruction,
    tools=fact_agent_tools, 
)


In [27]:
# make a copy of the NER agent's end state to use as the initial state for the fact agent
fact_agent_initial_state = ner_end_state.copy()

fact_agent_caller = await make_agent_caller(relevant_fact_agent, fact_agent_initial_state)

await fact_agent_caller.call("Propose fact types that can be found in the text.")
# await fact_agent_caller.call("Propose fact types that can be found in the text.", True)

session_end = await fact_agent_caller.get_session()

print("\n---\n")

print("\nSession state: ", session_end.state)

print("\nApproved entities: ", session_end.state.get(APPROVED_ENTITIES, []))

# Check that the agent proposed facts
if PROPOSED_FACTS in session_end.state:
    print("\nCorrectly proposed facts: ", session_end.state[PROPOSED_FACTS])
else:
    print("\nProposed facts not found in session state. What went wrong?")

# Check that the agent did not inappropriately approve facts
if APPROVED_FACTS in session_end.state:
    print("\nInappriately approved facts: ", session_end.state[APPROVED_FACTS])
else:
    print("\nApproved facts not found in session state, which is good.")


>>> User Query: Propose fact types that can be found in the text.
<<< Agent Response: Here are the general types of facts that could be extracted from the text, aligned with your goal (supply chain analysis / multi-level BOM context):

- Product, Reviews, Review
  - Captures that a Product has associated customer Reviews.
  - Useful for linking products to user feedback and identifying common issues or praise.

- Product, UsesMaterial, Material
  - Captures which Materials are used by a Product.
  - Helps trace BOM/components and material sourcing implications.

- Product, HasFinish, Finish
  - Captures the Finish applied to a Product (e.g., wood finish, paint).
  - Important for material processing, sourcing, and quality control.

- Product, HasProcessStep, ProcessStep
  - Captures manufacturing or assembly steps involved in making the Product.
  - Supports understanding of production flow and potential root-cause points in processes.

Notes:
- These fact types were proposed based on

In [28]:
await fact_agent_caller.call("Approve the proposed fact types.")

session_end = await fact_agent_caller.get_session()

print("Session state: ", session_end.state)

if APPROVED_FACTS in session_end.state:
    print("\nApproved fact types: ", session_end.state[APPROVED_FACTS])
else:
    print("\nFailed to approve fact types.")


>>> User Query: Approve the proposed fact types.
<<< Agent Response: Approval complete.

Approved fact types:
- (Product, Reviews, Review)
- (Product, UsesMaterial, Material)
- (Product, HasFinish, Finish)
- (Product, HasProcessStep, ProcessStep)

What this means:
- You can now extract facts from the approved files using these four relationship types between Product and the corresponding object types (Review, Material, Finish, ProcessStep).
- Each extracted fact will reflect a product-related piece of information such as a review, materials used, finish applied, or manufacturing/assembly steps.

Next steps (optional):
- I can run an extraction pass on the approved files and return a small set of example facts to illustrate the results.
- I can also build a simple mapping or schema outline showing how these fact types connect across multiple products (e.g., which materials and finishes are common across product families).
Session state:  {'approved_user_goal': {'kind_of_graph': 'supply